Minimum Cost to Cut a Stick
===
***
Problem
---
https://leetcode.com/problems/minimum-cost-to-cut-a-stick/

Given a wooden stick of length n units. The stick is labelled from 0 to n. For example, a stick of length 6 is labelled as follows:

[0, 1, 2, 3, 4, 5, 6]


Given an integer array cuts where cuts[i] denotes a position you should perform a cut at.
You should perform the cuts in order, you can change the order of the cuts as you wish.
The cost of one cut is the length of the stick to be cut, the total cost is the sum of costs of all cuts. When you cut a stick, it will be split into two smaller sticks (i.e. the sum of their lengths is the length of the stick before the cut). Please refer to the first example for a better explanation.

Return the minimum total cost of the cuts.


**Example 1:**  

Input: n = 7, cuts = [1,3,4,5]  
Output: 16  
Explanation: Using cuts order = [1, 3, 4, 5] as in the input leads to the following scenario:  
The first cut is done to a rod of length 7 so the cost is 7. The second cut is done to a rod of length 6 (i.e. the second part of the first cut), the third is done to a rod of length 4 and the last cut is to a rod of length 3. The total cost is 7 + 6 + 4 + 3 = 20.
Rearranging the cuts to be [3, 5, 1, 4] for example will lead to a scenario with total cost = 16 (as shown in the example photo 7 + 4 + 3 + 2 = 16).


**Example 2:**

Input: n = 9, cuts = [5,6,1,4,2]  
Output: 22  
Explanation: If you try the given cuts ordering the cost will be 25.  
There are much ordering with total cost <= 25, for example, the order [4, 6, 5, 2, 1] has total cost = 22 which is the minimum possible.
 

**Constraints:**  

2 <= n <= 10^6  
1 <= cuts.length <= min(n - 1, 100)  
1 <= cuts[i] <= n - 1  
All the integers in cuts array are distinct.  

In [ ]:
import functools
import random

**Brute force solution - \$O(k!)\$ where k is the number of cuts**

In [100]:
@functools.lru_cache(10000)
def min_cost_bf(n, cuts):
    if not cuts:
        return 0
    mincost = None
    for c in cuts:
        l = tuple(x for x in cuts if x < c)
        r = tuple(x - c for x in cuts if x > c)
        this_cost = n + cost(c, l) + cost(n - c, r)
        if mincost is None or this_cost < mincost:
            mincost = this_cost
    return mincost

**Bottom-up dynamic programming - \$O(k^3)\$ where k is the number of cuts**

In [111]:
def min_cost(n, cuts):
    cuts.sort()
    N = len(cuts) + 2
    table = [[0] * N for _ in range(N)]

    # Let `lenghts` be the array of all endpoints of the pieces
    # resulting after stick of length == n is chopped w.r.t `cuts`.
    # 
    # For each pair (i, i + j) in `lengths`, such that
    # j > 1, calculate the cost of cutting the stick[i:i + j]
    # The cost of cutting stick[i: i + j] is equal to:
    #
    #     j + min(stick[i:k] + stick[k:j] for k in [i + 1, j - 1])
    #
    # Note:
    # The cost of stick[i:i + 1] == 0, since no possible cuts
    # exists in lengths[i:i + 1]
    #
    # Finally return stick[0:n]
    lengths = [0] + cuts + [n]
    j = 2
    while j < N:
        for i in range(N - j):
            c = table[i][i + 1] + table[i + 1][i + j]
            for k in range(i + 1, i + j):
                c = min(c, table[i][k] + table[k][i + j])
            c += lengths[i + j] - lengths[i]
            table[i][i + j] = c
        j += 1
    return table[0][-1]

**Tests**

In [112]:
assert(min_cost(0, []) == 0)
assert(min_cost(10, []) == 0)
assert(min_cost(2, [1]) == 2)
assert(min_cost(3, [1]) == 3)
assert(min_cost(3, [2]) == 3)
assert(min_cost(3, [1, 2]) == 5)
assert(min_cost(4, [1]) == 4)
assert(min_cost(4, [1, 2]) == 6)
assert(min_cost(4, [1, 3]) == 7)


for _ in range(1000):
    n = random.randint(2, 30)
    cuts = list(set(random.randint(1, n - 1) for _ in range(n)))
    assert(cost(n, tuple(cuts)) == min_cost(n, cuts))